In [1]:
# !pip install pymongo langchain-groq

In [2]:
from pymongo import MongoClient
import os

from sentence_transformers import SentenceTransformer
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

c:\Users\Santanu\projects\the-communist-bot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
atlas_connection_string = os.getenv('ATLAS_CONNECTION_STRING')
atlas_cluster_password = os.getenv('ATLAS_CLUSTER_PASSWORD')
MONGODB_ATLAS_CLUSTER_URI = atlas_connection_string.replace("<password>", atlas_cluster_password)

# initialize MongoDB python client
client = MongoClient(MONGODB_ATLAS_CLUSTER_URI)

In [4]:
DB_NAME = "the-communist-bot"
COLLECTION_NAME = "manifesto"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "communist-manifesto"

MONGODB_COLLECTION = client[DB_NAME][COLLECTION_NAME]

In [6]:
type(MONGODB_COLLECTION)

pymongo.collection.Collection

In [9]:
def generate_embeddings(text: str) -> list[float]:
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    embeddings = model.encode(text)

    return embeddings.tolist()

In [10]:
def vector_search(user_query, collection):
    """
    Perform a vector search in the MongoDB collection based on the user query.

    Args:
    user_query (str): The user's query string.
    collection (MongoCollection): The MongoDB collection to search.

    Returns:
    list: A list of matching documents.
    """

    # Generate embedding for the user query
    query_embedding = generate_embeddings(user_query)

    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    # Define the vector search pipeline
    pipeline = [
        {
            "$vectorSearch": {
                "index": "communist-manifesto",  # Search index name
                "queryVector": query_embedding,  # Embedding representation of the use query
                "path": "embedding",             # Document field containing the embeddings
                "numCandidates": 150,            # Number of candidate matches to consider (Limits on the number of results to return)
                "limit": 15,                      # Return top 2 matches
            }
        },
        {
            "$project": {
                "_id": 0,                                 # Exclude the _id field
                "documentID": 1,                          # Include the documentID field
                # "file_path": 1,                           # Include the file_path field
                # "file_name": 1,                           # Include the file_name field
                # "page_number": 1,
                "content": 1,
                # "embedding": 1,
                # "score": {"$meta": "vectorSearchScore"},  # Include the search score
            }
        },
    ]

    # Execute the search
    results = collection.aggregate(pipeline)
    print(type(results))
    return list(results)

In [11]:
relevant_contexts = vector_search("Who are Proletarians?", MONGODB_COLLECTION)

c:\Users\Santanu\projects\the-communist-bot\venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


<class 'pymongo.command_cursor.CommandCursor'>


In [12]:
relevant_contexts

[{'documentID': '7d6d0282-eec7-4b73-9f13-3c20d97c233c',
  'content': 'II. Proletarians and Communists \nIn what relation do the Communists stand to the proletarians as a whole?  \nThe Communists do not form a separate party opposed to the other working -class parties.  \nThey have no interests separate and apart from those of  the proletariat as a whole.  \nThey do not set up any sectarian principles of their own, by which to shape and mould the \nproletarian movement.  \nThe Communists are distinguished from the other working- class parties by this only: 1. In the \nnational struggles of the proletarians of the different countries, they point out and bring to the \nfront the common interests of the entire proletariat, independently of all nationality. 2. In the \nvarious stages of development which the struggle of the working class against the bourgeoisie has \nto pass through, they always and everywhere represent the interests of the movement as a whole.'},
 {'documentID': 'f93d0913-

## Chatbot

In [13]:
groq_api_key = os.getenv('GROQ_API_KEY')

In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_community.llms import Ollama

In [15]:
model = ChatGroq(model_name="mixtral-8x7b-32768")

In [16]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

In [17]:
# Define your desired data structure.
class ModelOutput(BaseModel):
    response: str = Field(description="the response to the query")
    documentID: list[str] = Field(description="documentIDs of documents used to form response")

In [18]:
# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=ModelOutput)

In [19]:
# Creating the prompt
examples = [
    {"question": "Who is the current best F1 Driver?", 
     "context": "[{'documentID': '<Document ID>', 'file_path': '<File Path>', 'file_name': '<File Name>', 'page_number': '<Page Number>', 'embedding': '<Embedding>', 'score': '<Score>'}]",
     "answer": "{'response': 'The current best F1 driver is Fernando Alonso', 'documentID': ['d381adfb-d0b6-46ef-8e00-1c44c90ac14b', 'd381adfb-d0b6-46ef-8e00-1c44c90ac39b']}",
    },
    {"question": "Name four F1 Drivers", 
     "context": "[{'documentID': '<Document ID>', 'file_path': '<File Path>', 'file_name': '<File Name>', 'page_number': '<Page Number>', 'embedding': '<Embedding>', 'score': '<Score>'}]",
     "answer": "{'response': 'Four F1 drivers are:\n Max Verstappen\n Fernando Alonso\n Lewis Hamilton\n Charles LeClark', 'documentID': ['d381thfb-d0b6-46ef-8e00-1c44c90ac14b', 'd381adfb-d0b6-47sf-8e00-1c44c90ac39b']}",
    },
    {"question": "Who is the founder of Ferrari?", 
     "context": "[{'documentID': '<Document ID>', 'file_path': '<File Path>', 'file_name': '<File Name>', 'page_number': '<Page Number>', 'embedding': '<Embedding>', 'score': '<Score>'}]",
     "answer": "{'response': 'I do not know the answer to your question', 'documentID': []}",
    },
]

In [20]:
bot_instructions = """1. Respond with 'I do not know the answer to your question' if the relevant answer is not in the context
                      2. Do not form answers on your own. If an answer is not found on the context reply with 'I do not know the answer to your question'
                      3. Mention the documentIDs of the contexts where you fetched information from
                      4. REMEMBER not to form answers of your own
                      5. If the answer is not in the relevant context, keep citation as empty list
                      6. You can include bullet points wherever necessary
                      7. Give a detailed answer but only include information from the context provided
                      8. Explaining the documentIDs is not required at all
                      9. All your answer should go within 'response'. There should STRICTLY be no content after 'documentID' is mentioned
                      10. Do not include information based on your own knowledge, reply with 'I do not know the answer to your question' if the answer is not in context
"""

In [21]:
# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}\n{context}"),
        ("ai", "{answer}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())

Human: Who is the current best F1 Driver?
[{'documentID': '<Document ID>', 'file_path': '<File Path>', 'file_name': '<File Name>', 'page_number': '<Page Number>', 'embedding': '<Embedding>', 'score': '<Score>'}]
AI: {'response': 'The current best F1 driver is Fernando Alonso', 'documentID': ['d381adfb-d0b6-46ef-8e00-1c44c90ac14b', 'd381adfb-d0b6-46ef-8e00-1c44c90ac39b']}
Human: Name four F1 Drivers
[{'documentID': '<Document ID>', 'file_path': '<File Path>', 'file_name': '<File Name>', 'page_number': '<Page Number>', 'embedding': '<Embedding>', 'score': '<Score>'}]
AI: {'response': 'Four F1 drivers are:
 Max Verstappen
 Fernando Alonso
 Lewis Hamilton
 Charles LeClark', 'documentID': ['d381thfb-d0b6-46ef-8e00-1c44c90ac14b', 'd381adfb-d0b6-47sf-8e00-1c44c90ac39b']}
Human: Who is the founder of Ferrari?
[{'documentID': '<Document ID>', 'file_path': '<File Path>', 'file_name': '<File Name>', 'page_number': '<Page Number>', 'embedding': '<Embedding>', 'score': '<Score>'}]
AI: {'response': 

In [22]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You're an expert AI assistant specializing in advanced question answering.\
         Generate answers STRICTLY following the below instructions: \n{bot_instructions}\
         The answer format should STRICTLY follow {format_instructions}\
         Do not add extra information outside of the format"),
        few_shot_prompt,
        ("human", "{question}\n{context}"),
    ]
)

In [30]:
chain = final_prompt | model

response = chain.invoke({"question": "What is communism?", "context": relevant_contexts, "bot_instructions": bot_instructions, "format_instructions": parser.get_format_instructions()})

In [31]:
response.content

"{'response': 'Communism is a political and economic ideology that aims to create a classless, stateless society in which all property is publicly owned, and each person works and is paid according to their abilities and needs. It is based on the idea of common ownership and the elimination of social classes. Communism emphasizes social concern for all members of society rather than just the wealthy few. It is contrasted with capitalism, which is based on private or corporate ownership and operation for profit.', 'documentID': ['7d6d0282-eec7-4b73-9f13-3c20d97c233c', 'f93d0913-d16b-42c4-9373-8a6094e9f8f8', '04660687-8f11-48fe-a7db-875b52118dbb', '3dac08f8-5d94-41ee-b313-a77a2e34fe4d', 'd381adfb-d0b6-46ef-8e00-1c44c90ac14b', 'e11ebce6-c481-4727-8b84-dfa8dfbef45f', '53b8ab22-e9c5-4d95-89e5-e5fcd2015e03', '71bbfec8-d6c8-41ad-860c-101d850c069e', '1f11f8d0-7151-402b-a37c-b3e43a81ba7a', '210a2150-0555-4f98-8918-a40ff0039bb0', 'f30a1cd9-1292-4917-bff6-0d58a18383e3', 'd162d7e1-cbe7-4840-995b-2

In [21]:
# relevant_contexts